# PyTorch BYOS

## Pre-requisites

This notebook shows how to use the SageMaker Python SDK to run your code in a local container before deploying to SageMaker's managed training or hosting environments.  This can speed up iterative testing and debugging while using the same familiar Python SDK interface.  Just change your estimator's `train_instance_type` to `local` (or `local_gpu` if you're using an ml.p2 or ml.p3 notebook instance).

In order to use this feature you'll need to install docker-compose (and nvidia-docker if training with a GPU).

**Note, you can only run a single local notebook at one time.**

In [ ]:
!/bin/bash ./utils/setup.sh

## Overview

The **SageMaker Python SDK** helps you deploy your models for training and hosting in optimized, productions ready containers in SageMaker. The SageMaker Python SDK is easy to use, modular, extensible and compatible with TensorFlow, MXNet, PyTorch and Chainer. This tutorial focuses on how to create a convolutional neural network model to train the [Cifar10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) using **PyTorch in local mode**.

### Set up the environment

This notebook was created and tested on a single ml.p2.xlarge notebook instance.

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the sagemaker.get_execution_role() with appropriate full IAM role arn string(s).

In [1]:
import os
import sagemaker
print(sagemaker.__version__)

from sagemaker.pytorch import PyTorch
from sagemaker.pytorch.model import PyTorchModel

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-consert-roberta'

role = sagemaker.get_execution_role()

2.112.2


In [2]:
import subprocess

instance_type = 'local'

if subprocess.call('nvidia-smi') == 0:
    ## Set type to GPU if one is present
    instance_type = 'local_gpu'
    
print("Instance type = " + instance_type)

Fri Oct 21 01:10:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   30C    P0    22W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Upload the data
We use the ```sagemaker.Session.upload_data``` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use this later when we start the training job.

In [7]:
WORK_DIRECTORY ='./STSdata'
print('s3://{}/{}'.format(bucket, prefix))
data_location = sagemaker_session.upload_data(WORK_DIRECTORY, key_prefix=prefix+'/datasets/STS')
print(data_location)
inputs = {'STS': data_location}
print(inputs)

s3://sagemaker-us-west-2-064542430558/sagemaker/DEMO-pytorch-consert-roberta
s3://sagemaker-us-west-2-064542430558/sagemaker/DEMO-pytorch-consert-roberta/datasets/STS
{'STS': 's3://sagemaker-us-west-2-064542430558/sagemaker/DEMO-pytorch-consert-roberta/datasets/STS'}


## Script Functions

SageMaker invokes the main function defined within your training script for training. When deploying your trained model to an endpoint, the model_fn() is called to determine how to load your trained model. The model_fn() along with a few other functions list below are called to enable predictions on SageMaker.

### [Predicting Functions](https://github.com/aws/sagemaker-pytorch-containers/blob/master/src/sagemaker_pytorch_container/serving.py)
* model_fn(model_dir) - loads your model.
* input_fn(serialized_input_data, content_type) - deserializes predictions to predict_fn.
* output_fn(prediction_output, accept) - serializes predictions from predict_fn.
* predict_fn(input_data, model) - calls a model on data deserialized in input_fn.

The model_fn() is the only function that doesn't have a default implementation and is required by the user for using PyTorch on SageMaker. 

## Create a training job using the sagemaker.PyTorch estimator

The `PyTorch` class allows us to run our training function on SageMaker. We need to configure it with our training script, an IAM role, the number of training instances, and the training instance type. For local training with GPU, we could set this to "local_gpu".  In this case, `instance_type` was set above based on your whether you're running a GPU instance.

After we've constructed our `PyTorch` object, we fit it using the data we uploaded to S3. Even though we're in local mode, using S3 as our data source makes sense because it maintains consistency with how SageMaker's distributed, managed training ingests data.


## SageMaker Training using GPU instance

In [8]:
# git_config = {'repo': 'https://github.com/HaoranLv/mae.git', 'branch': 'main'}

hyperparameters = {'train_data': 'stssick', 
                   'seed': 42,
                   'max_seq_length': 128,
                   'evaluation_steps': 2500, 
                   'apex_amp_opt_level':'01',
                   'cl_rate': 0.15,
                   'temperature': 0.1,
                   'learning_rate': 0.0000005,
                   'da_final_1': 'feature_cutoff',
                   'da_final_2': 'shuffle',
                   'cutoff_rate_final_1': 0.2,
                   'patience': 1000,
                   'model_name_or_path': 'roberta-base', 
                   'model_save_path': '/opt/ml/model/roberta',
                   'batch_size': 32, 
                   'num_epochs': 1} 
instance_type = 'ml.p3.2xlarge'
estimator = PyTorch(entry_point='main.py',
                            source_dir='./code/ConSert',
                            role=role,
                            hyperparameters=hyperparameters,
                            framework_version='1.7.1',
                            py_version='py36',
                            script_mode=True,
                            instance_count=1,  # 1 or 2 or ...
                            instance_type=instance_type)

estimator.fit(inputs)

2022-10-21 03:53:12 Starting - Starting the training job...
2022-10-21 03:53:42 Starting - Preparing the instances for trainingProfilerReport-1666324392: InProgress
.........
2022-10-21 03:55:11 Downloading - Downloading input data...
2022-10-21 03:55:41 Training - Downloading the training image....................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-10-21 03:58:50,701 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-10-21 03:58:50,732 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-10-21 03:58:50,740 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-10-21 03:58:51,502 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.6 -m pip install -r requirements.txt
  Created wheel for sentence-transformers: filename=sentence_transformers-0.

In [9]:
training_job_name = estimator.latest_training_job.name
# training_job_name = 'pytorch-training-2022-03-18-05-39-29-920'
print(training_job_name)

pytorch-training-2022-10-21-03-53-10-774


# Deploy the trained model to prepare for predictions

The deploy() method creates an endpoint (in this case locally) which serves prediction requests in real-time.

In [11]:
!rm -rf model.tar.gz
!rm -rf consert_model
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz .
!mkdir consert_model
!tar -xvf model.tar.gz -C ./consert_model

download: s3://sagemaker-us-west-2-064542430558/pytorch-training-2022-10-21-03-53-10-774/output/model.tar.gz to ./model.tar.gz
roberta/
roberta/stsb_corr.pdf
roberta/modules.json
roberta/args.json
roberta/0_Transformer/
roberta/0_Transformer/sentence_bert_config.json
roberta/0_Transformer/merges.txt
roberta/0_Transformer/vocab.json
roberta/0_Transformer/pytorch_model.bin
roberta/0_Transformer/config.json
roberta/0_Transformer/tokenizer_config.json
roberta/0_Transformer/special_tokens_map.json
roberta/command.txt
roberta/sts_eval_last2/
roberta/sts_eval_last2/similarity_evaluation_sts-14-headlines_results.csv
roberta/sts_eval_last2/similarity_evaluation_sts-16_results.csv
roberta/sts_eval_last2/similarity_evaluation_sts-14_results.csv
roberta/sts_eval_last2/STS13-results.json
roberta/sts_eval_last2/similarity_evaluation_sts-14-OnWN_results.csv
roberta/sts_eval_last2/summary.json
roberta/sts_eval_last2/similarity_evaluation_sts-15-answers-forums_results.csv
roberta/sts_eval_last2/STS15-r

# Finetune

In [12]:
WORK_DIRECTORY1 ='./finetune'
print('s3://{}/{}'.format(bucket, prefix))
data_location = sagemaker_session.upload_data(WORK_DIRECTORY1, key_prefix=prefix+'/datasets/finetune')
WORK_DIRECTORY2 ='./consert_model/roberta'
model_location = sagemaker_session.upload_data(WORK_DIRECTORY2, key_prefix=prefix+'/models/consert_model')
print(model_location)
inputs = {'training': data_location, 'pretrained_model': model_location}

s3://sagemaker-us-west-2-064542430558/sagemaker/DEMO-pytorch-consert-roberta
s3://sagemaker-us-west-2-064542430558/sagemaker/DEMO-pytorch-consert-roberta/models/consert_model


In [14]:
# git_config = {'repo': 'https://github.com/HaoranLv/mae.git', 'branch': 'main'}

hyperparameters = {'model_path': "/opt/ml/input/data/pretrained_model/0_Transformer", 
                   'label_list': "/opt/ml/input/data/training/labels_list.json",
                   'train_path': "/opt/ml/input/data/training/train.jsonl",
                   'dev_path': "/opt/ml/input/data/training/dev.jsonl", 
                   'learning_rate': 3e-5,
                   'warmup_ratio': 0.06,
                   'max_length': 128,
                   'model_save': '/opt/ml/model',
                   'batch_size': 32, 
                   'epochs': 10} 
instance_type = 'ml.p3.2xlarge'
estimator = PyTorch(entry_point='main.py',
                            source_dir='./code/code',
                            role=role,
                            hyperparameters=hyperparameters,
                            framework_version='1.10.2',
                            py_version='py38',
                            script_mode=True,
                            instance_count=1,  # 1 or 2 or ...
                            instance_type=instance_type)

estimator.fit(inputs)

2022-10-21 05:18:12 Starting - Starting the training job...
2022-10-21 05:18:38 Starting - Preparing the instances for trainingProfilerReport-1666329492: InProgress
............
2022-10-21 05:20:26 Downloading - Downloading input data...
2022-10-21 05:21:06 Training - Downloading the training image..........................bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-10-21 05:25:24,812 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-10-21 05:25:24,841 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-10-21 05:25:24,851 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-10-21 05:25:25,541 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.8 -m pip install -r requirements.txt
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 50.1 MB/s eta 0

# Deploy

In [15]:
training_job_name = estimator.latest_training_job.name
# training_job_name='pytorch-training-2022-10-19-06-18-28-141'

In [16]:
!rm -rf model.tar.gz
!rm -rf finetune_model
!mkdir finetune_model
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz .
!tar -xvf model.tar.gz -C ./finetune_model

download: s3://sagemaker-us-west-2-064542430558/pytorch-training-2022-10-21-05-18-11-669/output/model.tar.gz to ./model.tar.gz
model.bin


In [17]:
!rm -rf model-roberta.tar.gz
!mv ./finetune_model/model.bin ./code/code/model.bin
!cd code && tar -czvf ../model-roberta.tar.gz code
!aws s3 cp model-roberta.tar.gz s3://$bucket/$training_job_name/output/model-roberta.tar.gz

code/
code/.ipynb_checkpoints/
code/.ipynb_checkpoints/requirements-checkpoint.txt
code/.ipynb_checkpoints/main-checkpoint.py
code/.ipynb_checkpoints/inference-checkpoint.py
code/.ipynb_checkpoints/eval-checkpoint.py
code/__pycache__/
code/__pycache__/main.cpython-38.pyc
code/model.bin
code/requirements.txt
code/eval.py
code/main.py
code/inference.py
upload: ./model-roberta.tar.gz to s3://sagemaker-us-west-2-064542430558/pytorch-training-2022-10-21-05-18-11-669/output/model-roberta.tar.gz


In [18]:
# instance_type = 'local'
instance_type = 'ml.g4dn.xlarge'

# predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type)

pytorch_model = PyTorchModel(model_data='s3://{}/{}/output/model-roberta.tar.gz'.format(bucket, training_job_name), role=role,
                             entry_point='inference.py', framework_version='1.7.1', py_version='py36', model_server_workers=1)  # TODO set model_server_workers=1 to avoid torchhub bug
predictor = pytorch_model.deploy(instance_type=instance_type, initial_instance_count=1)

----------!

# Invoking the endpoint

In [19]:
import sagemaker
import json
sm = sagemaker.Session().sagemaker_runtime_client

input_data = {'text':"An article is a word that comes before a noun to show if it's specific or general."}

endpoint_name='pytorch-inference-2022-10-21-05-53-30-876'
response = sm.invoke_endpoint(
    EndpointName=endpoint_name, Body=json.dumps(input_data), ContentType="application/json"
)

result = response["Body"].read().decode()

# result = list(map(float, result[1:-1].split()))
eval(result)

{'res': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

# Clean-up

Deleting the local endpoint when you're finished is important since you can only run one local endpoint at a time.

In [ ]:
# estimator.delete_endpoint()
predictor.delete_endpoint()
serverless_predictor.delete_endpoint()